<a href="https://colab.research.google.com/github/C-Gabs/Challenge-Robot-Trading-Alura/blob/main/Robot_Trading_Completo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Configurción del ambiente

In [44]:
#!pip install plotly==5.24.0
#!pip install pandas==2.2.2
#!pip install numpy==1.26.4
#!pip install matplotlib==3.7.1
#!pip install yfinance==0.2.44

In [58]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from urllib.request import urlopen
import datetime
import yfinance as yf
from IPython.display import clear_output
import time
import plotly.express as px
import warnings

!python -V
print('------')
!pip show Pandas | grep 'Name\|Version'
print('------')
!pip show Numpy | grep 'Name\|Version'
print('------')
!pip show Matplotlib | grep 'Name\|Version'
print('------')
!pip show plotly | grep 'Name\|Version'
print('------')
!pip show bs4 | grep 'Name\|Version'
print('------')
!pip show urllib | grep 'Name\|Version'
print('------')
!pip show datetime | grep 'Name\|Version'
print('------')
!pip show time | grep 'Name\|Version'
print('------')
!pip show yfinance | grep 'Name\|Version'

global df_bitcoin, precio_actual, tendencia, media_bitcoin,  algoritmo_decision

Python 3.12.11
------
Name: pandas
Version: 2.2.2
------
Name: numpy
Version: 2.0.2
Name: lapack-lite
Name: dragon4
Name: libdivide
Name: Meson
Name: spin
Name: OpenBLAS
Name: LAPACK
Name: GCC runtime library
Version 3.1, 31 March 2009
                       Version 3, 29 June 2007
  5. Conveying Modified Source Versions.
  14. Revised Versions of this License.
Name: libquadmath
------
Name: matplotlib
Version: 3.10.0
------
Name: plotly
Version: 5.24.1
------
------
------
------
------
Name: yfinance
Version: 0.2.65


#2. Extraer los datos

## Obteniendo el historico del bitcoin con yfinance

In [46]:
def importar_base_bitcoin():
  global df_bitcoin, precio_actual, tendencia, media_bitcoin
  #Establecemos el intervalo de fechas en el que queremos obtener los datos
  fecha_actual = datetime.datetime.now()
  fecha_inicio = datetime.datetime.now() - datetime.timedelta(days = 7)
  #obtenemos los datos en un dataframe usando la api yfinance
  df_bitcoin = yf.download('BTC-USD', start = fecha_inicio, end=fecha_actual ,interval='5m')


##Obteniendo el precio actual y la tendecia del bitcoin con Web Scraping

In [47]:
def extraer_tendencias():
  global precio_actual, tendencia
#Usamos beatifulsoup para obtener los datos de coinmarketcap
  url = 'https://coinmarketcap.com/es/'
  response = urlopen(url)
  html = response.read()
  soup = BeautifulSoup(html, 'html.parser')
#Extraemos una lista con las etiquetas de cada moneda en la página
  lista = soup.find_all('tr',style="cursor:pointer")
#Iteramos en la lista hasta obtener la etiqueta con la información dee bitcoin
  for i in lista:
    symbol = i.find('p', class_= 'sc-65e7f566-0 byYAWx coin-item-symbol').getText()
    if symbol == 'BTC':
      td = i.find('td',style="text-align:end")
      #Extraemos el precio del bitcoin y limpiamos el texto para poder convertirlo a float
      precio_actual = float(td.div.span.getText().replace('$','').replace(',',''))
      break
#Buscamos la etiqueta que contiene la tedencia de crecimiento del bitcoin
#Como la clase varia en 2 opciones diferentes segun si la tendencia es de bajada o subida, implementamos una estructura condicional
#Con ayuda de una estructura condiconal determinamos si la tendencia es alcista o bajista
#Lo hacemos verificando el nombre de la clase de la etiqueta span que lo contiene
  if td.findNextSibling().span.span['class'][0] == 'icon-Caret-up':
    tendencia = 'Alta'
  elif td.findNextSibling().span.span['class'][0] == 'icon-Caret-down':
    tendencia = 'Baja'


#3. Limpieza de datos

In [48]:
def limpieza_datos():
  global media_bitcoin, df_bitcoin
  outliers = []
  #El método yf.download genera una columna multinivel, la eliminamos
  df_bitcoin.columns = df_bitcoin.columns.droplevel('Ticker')
  #Convertimos la zona horaria a la nuestra, en est caso es UTC-4
  df_bitcoin.index = df_bitcoin.index.tz_convert('America/Caracas')
  #Renombramos el indice
  df_bitcoin.rename_axis('Fecha',inplace=True)
  #Creamos una copia del dataframe para trabajar en el
  df_bitcoin_limpio = df_bitcoin.copy()
  #Eliminamos los indices duplicados
  df_bitcoin_limpio = df_bitcoin_limpio[~df_bitcoin_limpio.index.duplicated(keep='first')]
  #Reseteamos el indice para trabajar el formato de las fechas en una columna
  df_bitcoin_limpio.reset_index(inplace=True)
  #Formateamos las fechas
  df_bitcoin_limpio['Fecha'] = df_bitcoin_limpio['Fecha'].dt.strftime("%d/%m/%Y %H:%M")
  #Volvemos a establecer las fechas como indice
  df_bitcoin_limpio.set_index('Fecha',inplace=True)
  #Ejecutamos el metodo describe para observar la información con la que vamos a trabajar
  df_bitcoin_limpio.describe()
  #Mostramos un grafico boxplot para identificar los outliers
  '''plt.boxplot(df_bitcoin_limpio['Close'])
  plt.title('Datos sin limpiar')
  plt.grid(axis='y')
  plt.show()'''
  #Tratamos los posibles valores nulos en la columna "close" aplicando una interpolación lineal y asi mantener valores con sentido
  df_bitcoin_limpio['Close'] = df_bitcoin_limpio['Close'].interpolate(method='linear')
  #Eliminamos los valores menores o iguales a "cero" dentro de la columna "Volume"
  df_bitcoin_limpio = df_bitcoin_limpio[df_bitcoin_limpio['Volume'] > 0]
  #Hacemos un boxplots para ver si hay nuevos outliers
  #Obtenemos los outliers directamente del boxplot, esto tambien muestra un boxplot con los datos que estamos trabajando
  for flier in plt.boxplot(df_bitcoin_limpio['Close'])['fliers']: #Hacemos un ciclo for para explorar cada conjunto de outliers, superior e inferior
    #Almacenamos el conjunto en una variable auxiliar y la imprimimos en pantalla
    outliers = flier.get_ydata()
    '''print(f'Outliers: {outliers}')'''
    #Recorremos cada elemento del conjunto de outliers y lo eliminamos del dataframe
    for outlier in outliers:
      df_bitcoin_limpio = df_bitcoin_limpio[df_bitcoin_limpio['Close'] != float(outlier)]
  plt.close() #Cerramos la figura para que no se muestre en pantalla
  #Usamos el metodo quantile de pandas para calcular los quantiles
  Q1 = df_bitcoin_limpio['Close'].quantile(0.25)
  Q3 = df_bitcoin_limpio['Close'].quantile(0.75)
  #Imprimimos los quantiles en pantalla
  '''print(f'Q1: {Q1}, Q3: {Q3}')'''
  #Limpiamos el dataframe dejando solo los valores dentro de los quantiles
  df_bitcoin_limpio = df_bitcoin_limpio[(df_bitcoin_limpio['Close'] >= Q1) & (df_bitcoin_limpio['Close'] <= Q3)]
  #Mostramos en pantalla un nuevo grafico boxplot para ver los resultados
  '''plt.boxplot(df_bitcoin_limpio['Close'])
  plt.grid(axis='y')
  plt.title('Datos limpios')
  plt.show()'''
  #Obtenemos la media de los datos de cierre en la columna "Close"
  media_bitcoin = df_bitcoin_limpio['Close'].mean()
  media_bitcoin = media_bitcoin
  '''print(f'Media_bitcoin: {media_bitcoin}')'''

#4. Creando algoritmo de toma de desiciones

In [49]:
def tomar_decisiones():
  global df_bitcoin, precio_actual, tendencia, media_bitcoin,  algoritmo_decision, color_flecha
#Establecemos el algoritmo de desición
  if (precio_actual >= media_bitcoin) and tendencia == 'Baja':
    algoritmo_decision = 'Vender'
    color_flecha = 'red'
  elif (precio_actual < media_bitcoin) and tendencia == 'Alta':
    algoritmo_decision = 'Comprar'
    color_flecha = 'green'
  else:
    algoritmo_decision = 'Esperar'
    color_flecha = 'grey'

#5. Visualización

In [72]:
def visualizacion():
  global df_bitcoin, precio_actual, tendencia, media_bitcoin,  algoritmo_decision
#Establecemos una nueva columna con el precio promedio del bitcoin
  df_bitcoin['Promedio'] = media_bitcoin
#Reseteamos el indice para trabajar las fechas en una columna y facilitar la visualización
  df_bitcoin.reset_index(inplace=True)
#Establecemos los parametros de nuestro grafico y lo guardamos en una variable
  fig = px.line(df_bitcoin, x='Fecha', y=["Close",'Promedio'],hover_data={'Fecha': "|%d %B %Y %H:%M"},
                title='Precio historico del bitcoin (últimos 7 dias)',labels={'value':'Precio $ (BTC)'},
                template='plotly_dark')
#Añadimos indicadores extra al grafico, establecemos el formato que queremos para los mismos y añadimos una anotación que indica cuando vender, comprar o esperar

  fig.update_xaxes(tickformat="%d-%b-%Y",minor=dict(ticks='inside',showgrid=True),
                ticks= "inside",ticklen = 10,
                tick0 = datetime.datetime.today()-datetime.timedelta(days=7),
                rangeslider_visible=True,
                rangeselector=dict(
                activecolor='blue',  # Color de los botones activos
                bgcolor='gray',      # Color de fondo de los botones
                bordercolor='black', # Color del borde de los botones
                font=dict(color='white'), # Color de la fuente de los botones
                buttons=list([
                dict(count=1, label="1d", step="day", stepmode="backward"),
                dict(count=2, label="2d", step="day", stepmode="backward"),
                dict(count=3, label="3d", step="day", stepmode="backward"),
                dict(count=4, label="4d", step="day", stepmode="backward"),
                dict(count=5, label="5d", step="day", stepmode="backward"),
                dict(step="all")])),
                tickformatstops = [
                dict(dtickrange=[None, 7200000], value="%H:%M m"),
                dict(dtickrange=[7200000, 43200000], value="%H:%M h"),
                dict(dtickrange=[43200000, 86400000], value="%d-%b-%Y"),])
  fig.add_annotation(x=df_bitcoin['Fecha'].iloc[-1], y=df_bitcoin['Close'].iloc[-1], text = algoritmo_decision, showarrow=True, arrowhead=1,arrowsize=1,arrowwidth=2,arrowcolor=color_flecha)
#Mostramos el grafico en pantalla
  fig.show()

#6. Automatización

In [73]:
#Automatizamos el codigo para que se actualice cada 5 minutos
while(True):
  clear_output()
  importar_base_bitcoin()
  extraer_tendencias()
  limpieza_datos()
  tomar_decisiones()
  visualizacion()
  time.sleep(300)

/tmp/ipython-input-3174735811.py:7: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-385552113.py:22: DeprecationWarning:

Call to deprecated method findNextSibling. (Replaced by find_next_sibling) -- Deprecated since version 4.0.0.



KeyboardInterrupt: 